In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
import random

# Simulation Project (FA23):
We are looking to simulate bis in the workpalce




## Setup:

In [2]:
worker_id = 1

work_levels = {
    1 : "J",
    2 : "M",
    3 : "S",
    4 : "E"
}

worker_genders = {
    0 : "M",
    1 : "F"
}

level_sizes = {
    1 : 400,
    2 : 100,
    3 : 25,
    4 : 5
}

level_ext_stay_times = {
    1 : 0,
    2 : 10,
    3 : 20,
    4 : 30
}

gender_stay_times = {
    0 : 30,
    1 : 25
}

worker_db = {
    1 : [],
    2 : [],
    3 : [],
    4 : []
}

In [3]:
class worker(object):
    
    def __init__(self, level, gender, idx, start_time):
        self.level = level
        self.gender = gender
        self.idx = idx
        self.start_time = start_time
        
    def __lt__(self, other):
        return self.idx < other.get_index()
        
    def __str__(self):
        return "[level: %s, gender: %s, id: %i, end_time: %f]" \
            % (self.level, self.gender, self.idx, self.end_time)
        
    def get_index(self):
        return self.idx
    
    def get_level(self):
        return self.level
    
    def set_end_time(self):
        if self.gender == 0:
            self.end_time = self.start_time \
                + np.random.exponential( \
                gender_stay_times[self.gender] + level_ext_stay_times[self.level])
            
        else:
            self.end_time = self.start_time \
                + np.random.exponential( \
                gender_stay_times[self.gender] + level_ext_stay_times[self.level])
            
#                 + min(np.random.exponential( \
#                 gender_stay_times[self.gender] + level_ext_stay_times[self.level]), \
#                 gender_stay_times[self.gender] + level_ext_stay_times[self.level])
        
    def get_end_time(self):
        return self.end_time
    
    def get_gender(self):
        return self.gender

In [4]:
def hire_worker(level, idx):
    gender = round(math.floor(2*np.random.uniform()))

    hire = worker(level, gender, idx, time.time())
    hire.set_end_time()
    
    worker_db[hire.get_level()].append(hire)

def populate_workforce(start_id, all_male = False, all_female = False):
    
    for key in worker_db.keys():
        worker_db[key] = []
    
    level = 4
    idx = 1
    
    while level >= 1:
        for i in range(level_sizes[level]):
            hire_worker(level, idx)
        
            idx += 1 
            
        level -= 1
    return idx 

def update_workforce(start_id):
    # Delete expired workers:
    for level in worker_db.keys():
        
        l = len(worker_db[level])
        idx = 0
        
        while idx < l:
            if worker_db[level][idx].get_end_time() < time.time():
                worker_db[level] = worker_db[level][:idx] \
                                    + worker_db[level][idx+1:]
                l -= 1
            idx += 1
                
                
    # Promote to fill ranks:
                
    lvl = 4
    while level > 1:
        while len(worker_db[level]) < level_sizes[level]:
            worker_db[level].append(worker_db[level-1][0])
            worker_db[level-1] = worker_db[level-1][1:]
        level -= 1
            
    while len(worker_db[1]) < level_sizes[1]:
        hire_worker(level, start_id)
            
            
        

In [15]:
def gender_distribution(lst):
    count_male = 0
    count_female = 0
    total = 0
    
    for key in lst.keys():
        for wrkr in lst[key]:
            if wrkr.get_gender() == 0:
                count_male += 1
            else:
                count_female += 1
            total += 1
            
    pct_male = count_male
    pct_female = count_female
    
    return total

### Testing / Playground

In [6]:
# populate_workforce(worker_id)

# print([str(i) for i in worker_db[1]])

## Simulation:

In [16]:
state = 0

print("press enter to start!")

input()

worker_id = populate_workforce(worker_id)

state = 1

beg_time = time.time()


while state == 1:
    print("starting simulation")
    while time.time() <= beg_time + 5:
        update_workforce(worker_id)
    
    ans = input("Round complete. type 'yes' to continue.\n")
    
    print("\n")
    
    if ans != "yes":
        state = 0
        print("Simulation complete.")

        print("Stats:\n")

        print("population male: %f \n" % gender_distribution(worker_db))
        
    else: beg_time = time.time()
        
        
    

press enter to start!

starting simulation
Round complete. type 'yes' to continue.



Simulation complete.
Stats:

population male: 530.000000 

